In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("airbnb-listings.csv",sep=';')

/var/folders/f4/ck_1f6t90xndlxh8gvfjm79w0000gn/T/ipykernel_64581/838074011.py:1: DtypeWarning: Columns (0,2,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("airbnb-listings.csv",sep=';')


In [10]:
df.head(5)

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,4008728,https://www.airbnb.com/rooms/4008728,20170402075052,2017-04-02,"Luxurious 3 bedroom, centrum, 180m2",‪This luxurious apartment is situated in the c...,In the heart of hip & lively Amsterdam Centrum...,‪This luxurious apartment is situated in the c...,none,There is a wide variety of cafes and restauran...,...,10.0,10.0,9.0,NaN,Amsterdam,strict,1.0,1.52,"52.36523660409065, 4.878249575101092","Host Has Profile Pic,Host Identity Verified,Is..."
1,7778612,https://www.airbnb.com/rooms/7778612,20170402075052,2017-04-02,Luxury apartment in city centre,"Beautiful, large (105m2, 2 floors) and quiet a...",1 living room 1 dining room 2 bedrooms 1 bathr...,"Beautiful, large (105m2, 2 floors) and quiet a...",none,Within a few minutes walking you'll be in the ...,...,10.0,10.0,9.0,NaN,Amsterdam,strict,1.0,0.81,"52.367309030897516, 4.8738407410699915","Host Is Superhost,Host Has Profile Pic,Host Id..."
2,8264596,https://www.airbnb.com/rooms/8264596,20170402075052,2017-04-02,Cosy apartment across Vondelpark,This central located apartment will make you f...,NaN,This central located apartment will make you f...,none,NaN,...,10.0,10.0,10.0,NaN,Amsterdam,flexible,1.0,0.05,"52.361943953804776, 4.866686522141489","Host Has Profile Pic,Host Identity Verified,Is..."
3,2180729,https://www.airbnb.com/rooms/2180729,20170402075052,2017-04-02,Spacious City Apartment Oud-West,Living like an Amsterdam resident in this ligh...,There are 2 bedrooms each with a double bed. I...,Living like an Amsterdam resident in this ligh...,none,The 85 m2 apartment is located on a quiet stre...,...,9.0,9.0,9.0,NaN,Amsterdam,flexible,1.0,0.62,"52.37014616115532, 4.866281767968382","Host Has Profile Pic,Host Identity Verified,Is..."
4,14463171,https://www.airbnb.com/rooms/14463171,20170402075052,2017-04-02,Cosy Studio Apartment Center Amsterdam,Studio Apartment Centre Amsterdam Perfect stud...,NaN,Studio Apartment Centre Amsterdam Perfect stud...,none,NaN,...,8.0,9.0,9.0,NaN,Amsterdam,moderate,1.0,2.04,"52.368817829014425, 4.871249476925455",Host Has Profile Pic


In [11]:
df_dev = df.head(1000)

In [22]:
listing_titles = df_dev['Name'].tolist()
listing_Description = df['Description'].tolist()



In [18]:
len(listing_titles),len(set(listing_titles))


(1000, 1000)

In [23]:
len(listing_Description),len(set(listing_Description))

(494954, 484374)

In [8]:
from simhash import Simhash

def get_simhash(text):
    return Simhash(text).value

In [9]:
df_dev['Name'].fillna('', inplace=True)


/var/folders/f4/ck_1f6t90xndlxh8gvfjm79w0000gn/T/ipykernel_64046/2485416966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev['Name'].fillna('', inplace=True)


In [10]:
df_dev['Title_SimHash'] = df_dev['Name'].apply(get_simhash)


/var/folders/f4/ck_1f6t90xndlxh8gvfjm79w0000gn/T/ipykernel_64046/3028019436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev['Title_SimHash'] = df_dev['Name'].apply(get_simhash)


In [11]:
objs = [] # a list of (id, simhash)
f = 64



In [14]:
def hamming_distance(hash1, hash2):
    hash1 = int(hash1)
    hash2 = int(hash2)
    x = (hash1 ^ hash2) & ((1 << 64) - 1)
    tot = 0
    while x:
        tot += 1
        x &= x - 1
    return tot

# Threshold for considering items as duplicates
threshold = 12  # Example threshold

duplicates = []
for i in range(len(df_dev)):
    for j in range(i+1, len(df_dev)):
        hash1 = df_dev.iloc[i]['Title_SimHash']
        hash2 = df_dev.iloc[j]['Title_SimHash']
        if hamming_distance(hash1, hash2) <= threshold:
            duplicates.append((i, j))

In [15]:
indexes_to_remove = set([dup[1] for dup in duplicates])

df_cleaned = df_dev.drop(indexes_to_remove)

df_cleaned.reset_index(drop=True, inplace=True)

len(df_cleaned)


958

In [16]:
duplicates

[(17, 54),
 (17, 173),
 (20, 29),
 (33, 67),
 (33, 127),
 (39, 253),
 (44, 173),
 (96, 301),
 (106, 259),
 (106, 328),
 (125, 356),
 (127, 301),
 (129, 392),
 (134, 467),
 (153, 209),
 (158, 200),
 (197, 380),
 (198, 253),
 (218, 301),
 (238, 253),
 (251, 301),
 (253, 576),
 (258, 876),
 (301, 380),
 (310, 462),
 (314, 428),
 (323, 818),
 (328, 426),
 (503, 519),
 (507, 591),
 (539, 685),
 (550, 685),
 (608, 640),
 (608, 642),
 (608, 656),
 (608, 842),
 (631, 650),
 (638, 705),
 (638, 777),
 (638, 854),
 (640, 642),
 (640, 656),
 (640, 842),
 (656, 842),
 (686, 765),
 (751, 774),
 (767, 788),
 (767, 805),
 (774, 869),
 (788, 805),
 (788, 888),
 (792, 861),
 (805, 888),
 (830, 861),
 (837, 873),
 (841, 847),
 (888, 898)]

In [17]:
for i, j in duplicates:
    print("Duplicate Pair:")
    print("Row Index:", i)
    print(df_dev.iloc[i][''])
    print("\nRow Index:", j)
    print(df_dev.iloc[j])

Duplicate Pair:
Row Index: 17
ID                                                                          9672251
Listing Url                                    https://www.airbnb.com/rooms/9672251
Scrape ID                                                            20170402075052
Last Scraped                                                             2017-04-02
Name                                                  Light and spacious apartment.
                                                        ...                        
Calculated host listings count                                                  1.0
Reviews per Month                                                              0.37
Geolocation                                     52.3704416857012, 4.873528148594769
Features                          Host Has Profile Pic,Host Identity Verified,Is...
Title_SimHash                                                  11691680559576359662
Name: 17, Length: 90, dtype: object

Row Index

In [25]:
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zeyunhuang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
df['Description'] = df['Description'].astype(str)

In [30]:
df_dev['tokenized'] = df_dev['Description'].apply(word_tokenize)


/var/folders/f4/ck_1f6t90xndlxh8gvfjm79w0000gn/T/ipykernel_64581/2293293186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev['tokenized'] = df_dev['Description'].apply(word_tokenize)


In [51]:

# List of tokenized descriptions from df_dev
sentences = df['tokenized'].tolist()

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Example usage of the model
# vector = model.wv['example']  # Replace 'example' with a relevant word

# Save the model
model.save("word2vec_model.bin")


KeyboardInterrupt: 

In [28]:
from gensim.models import KeyedVectors

filepath = 'GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(filepath, binary=True)

In [77]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to convert sentences to vectors
def sentence_to_vector(sentence, model):
    words = word_tokenize(sentence)
    # Access vectors directly from the model
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)


# Preprocess and vectorize the input sentence
input_sentence = "2 bedroom, has kitchen"
input_vector = sentence_to_vector(input_sentence, model)

# Vectorize each description
description_vectors = df_dev['tokenized'].apply(lambda x: sentence_to_vector(' '.join(x), model))

# Calculate cosine similarity with the input sentence
similarities = [cosine_similarity([input_vector], [desc_vector])[0][0] for desc_vector in description_vectors]

# Combine similarities with descriptions
description_similarity = list(zip(df_dev['Description'], similarities))

# Sort by similarity
sorted_descriptions = sorted(description_similarity, key=lambda x: x[1], reverse=True)

# Display the most similar descriptions
for desc, similarity in sorted_descriptions[:5]:  # Adjust the number to display as needed
    print(f"Description: {desc}\nSimilarity: {similarity}\n")


Description: 2 bed 1  bathroom apartment
Similarity: 0.7711312770843506

Description: Spacious apartment,  15 minutes by bus from central station. The appartement living room (with sleeping bench, for 2 persons), kitchen, bedroom (with double bed), bathroom, toilet and balcony.
Similarity: 0.7678413987159729

Description: Charming vintage beach-style town home. 3 bedrooms+extra game room, pool out sofa, 2.5 bathrooms, Wifi, parking garage, TVs. Prime Woodland Hills location only 20 min. drive to Universal and 15 min. drive to Malibu beaches and walking distance to The Village and Westfield mall. Tile and wooden floors, huge remodeled kitchen/new appliances. Fire-place in living area and fire-place in master bedroom. Fully furnished, brand new washer, and dryer. Pool and spa in the complex. Gorgeous 2000 Sq.Ft. home. Front yard with small table and two chairs. Living room has TV and fireplace. L-shaped sofa and coffee table. 2 car garage plus washer and dryer in garage. Play room next t

In [3]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')

In [38]:
def encode_sentence(sentence):
    # Tokenize and encode the sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Get the embeddings
    with torch.no_grad():
        outputs = model_bert(**inputs)

    # Mean pooling - take attention mask into account for correct averaging
    input_mask_expanded = inputs['attention_mask'].unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
    sum_embeddings = torch.sum(outputs.last_hidden_state * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask

    return mean_embeddings[0]


In [13]:
df_dev['Description_Vector'] = df_dev['Description'].apply(lambda x: encode_sentence(x).numpy())


/var/folders/f4/ck_1f6t90xndlxh8gvfjm79w0000gn/T/ipykernel_64581/1177759404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev['Description_Vector'] = df_dev['Description'].apply(lambda x: encode_sentence(x).numpy())


In [67]:
from sklearn.metrics.pairwise import cosine_similarity

# Example usage
input_sentence = "convenient, beautiful view, near subway stations,close to airport"
input_vector = encode_sentence(input_sentence)

# Assuming df_dev['Description'] contains your descriptions
description_vectors = df_dev['Description_Vector']

# Calculate cosine similarity
similarities = []
for desc_vector in description_vectors:
    # Reshape the description vector to ensure it's 2D
    desc_vector_reshaped = desc_vector.reshape(1, -1)
    similarity = cosine_similarity([input_vector.numpy()], desc_vector_reshaped)[0][0]
    similarities.append(similarity)

# Rest of your code for sorting and displaying results

# Combine descriptions with their respective similarities
description_similarity = list(zip(df_dev['Description'], similarities))

# Sort by similarity in descending order
sorted_descriptions = sorted(description_similarity, key=lambda x: x[1], reverse=True)

# Display the top 5 most similar descriptions
print("Top 5 most similar descriptions:\n")
for desc, similarity in sorted_descriptions[:5]:
    print(f"Description: {desc}\nSimilarity Score: {similarity:.4f}\n")


Top 5 most similar descriptions:

Description: Cosy, quiet and modern apartment in a great neighborhood. Easy accessible. Multiple bar and restaurants around. Close to a canal. Convenient location, 10 min bike from Dam square. 55 square meters
Similarity Score: 0.8019

Description: 15 minutes from LAX airport; quite neighborhood; walking distance from shopping centers and a short bus ride away from Westfield Mall. Minutes from the (PHONE NUMBER HIDDEN) FWY's. Internet and cable available - personal bathroom.
Similarity Score: 0.7846

Description: 2 month rent Nice, furnished studio appartment. All luxuries needed. Big bed, shower, sofa, internet.  Neighbourhood Zurenborg is beautiful, with lots of nice café's and restaurants. Close to city center. Lot's of light! (URL HIDDEN)
Similarity Score: 0.7722

Description: great safe are near shopping.malls and beaches.public transportation
Similarity Score: 0.7657

Description: Lovely petite apartment. Great city hideout in a hip and diverse n

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming df_dev['Description'] contains your descriptions
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_dev['Description'])


In [51]:
def weighted_bert_embedding(sentence, model, tokenizer, vectorizer, tfidf_matrix):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Generate embeddings using the BERT model
    with torch.no_grad():
        outputs = model(**inputs)

    # Mean pooling
    input_mask_expanded = inputs['attention_mask'].unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
    sum_embeddings = torch.sum(outputs.last_hidden_state * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask

    # Retrieve the TF-IDF vector for the sentence
    sentence_tfidf = vectorizer.transform([sentence])
    words = sentence.split()

    # Weight the embeddings using TF-IDF
    weighted_embeddings = np.zeros(mean_embeddings.shape[1])
    for i, word in enumerate(words):
        if word in vectorizer.vocabulary_:
            tfidf_index = vectorizer.vocabulary_[word]
            weight = sentence_tfidf[0, tfidf_index]
            weighted_embeddings += weight * mean_embeddings[0].numpy()

    if np.sum(weighted_embeddings) != 0:
        return weighted_embeddings / len(words)
    else:
        return np.zeros(mean_embeddings.shape[1])

# Usage
input_vector = weighted_bert_embedding(input_sentence, model_bert, tokenizer, vectorizer, tfidf_matrix)

df_dev['Weighted_Description_Vector'] = df_dev['Description'].apply(
    lambda x: weighted_bert_embedding(x, model_bert, tokenizer, vectorizer, tfidf_matrix)
)



/var/folders/f4/ck_1f6t90xndlxh8gvfjm79w0000gn/T/ipykernel_64581/1206612157.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev['Weighted_Description_Vector'] = df_dev['Description'].apply(


In [75]:
from sklearn.metrics.pairwise import cosine_similarity

input_sentence = "convenient, beautiful view, near subway stations,close to airport, has kitchen,nightlife "
input_vector = encode_sentence(input_sentence)

# Calculate cosine similarity
weighted_description_vectors = df_dev['Weighted_Description_Vector']
# Assuming input_vector and the vectors in weighted_description_vectors are 1D numpy arrays
# Convert input_vector to a numpy array if it's a tensor
if isinstance(input_vector, torch.Tensor):
    input_vector = input_vector.numpy()

similarities = []
for desc_vector in weighted_description_vectors:
    # Convert desc_vector to a numpy array if it's a tensor
    if isinstance(desc_vector, torch.Tensor):
        desc_vector = desc_vector.numpy()

    # Ensure desc_vector is a numpy array and reshape it for cosine_similarity
    if isinstance(desc_vector, np.ndarray) and desc_vector.ndim == 1:
        desc_vector_reshaped = desc_vector.reshape(1, -1)
        input_vector_reshaped = input_vector.reshape(1, -1)
        similarity = cosine_similarity(input_vector_reshaped, desc_vector_reshaped)[0][0]
        similarities.append(similarity)
    else:
        # Handle cases where desc_vector is not in expected format
        similarities.append(0)  # or handle it as you see fit

# Rest of your code for sorting and displaying results
# ...



# Combine descriptions with their respective similarities
description_similarity = list(zip(df_dev['Description'], similarities))

# Sort by similarity in descending order
sorted_descriptions = sorted(description_similarity, key=lambda x: x[1], reverse=True)

# Display the top results
for desc, similarity in sorted_descriptions[:5]:
    print(f"Description: {desc}\nSimilarity Score: {similarity:.4f}\n")


Description: Cosy, quiet and modern apartment in a great neighborhood. Easy accessible. Multiple bar and restaurants around. Close to a canal. Convenient location, 10 min bike from Dam square. 55 square meters
Similarity Score: 0.8596

Description: 2 month rent Nice, furnished studio appartment. All luxuries needed. Big bed, shower, sofa, internet.  Neighbourhood Zurenborg is beautiful, with lots of nice café's and restaurants. Close to city center. Lot's of light! (URL HIDDEN)
Similarity Score: 0.8530

Description: Lovely petite apartment. Great city hideout in a hip and diverse neighbourhood. In 20 minutes you are in the city centre, by bike. By train it's only 10!  The appartment is renovated not long ago, spacious bedroom and cosy bathroom with Great shower!
Similarity Score: 0.8354

Description: The renovation of the house has just finished and it's ready to receive guests!  Fully renovated appartment in a quiet but centrally located neighbourhood. Walking distance from trainstati